In [ ]:
# # 下載資料
# !gdown --id '14CqX3OfY9aUbhGp4OpdSHLvq2321fUB7' --output data.zip
# # 解壓縮
# !unzip -qq -u data.zip
# # 確認目前的檔案
# !ls

In [179]:
import os
# 讀取 label.csv
import pandas as pd
# 讀取圖片
from PIL import Image
from PIL import ImageFilter
import numpy as np

import torch
# Loss function
from torchvision.utils import save_image
import torch.nn.functional as F
# 讀取資料
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
# 載入預訓練的模型
import torchvision.models as models
# 將資料轉換成符合預訓練模型的形式
import torchvision.transforms as transforms
# 顯示圖片
import matplotlib.pyplot as plt

device = torch.device("cuda")

## 讀取資料庫

In [200]:
# 實作一個繼承 torch.utils.data.Dataset 的 Class 來讀取圖片
class Adverdataset(Dataset):
    def __init__(self, root, label, transforms):
        # 圖片所在的資料夾
        self.root = root
        # 由 main function 傳入的 label
        self.label = torch.from_numpy(label).long()
        # 由 Attacker 傳入的 transforms 將輸入的圖片轉換成符合預訓練模型的形式
        self.transforms = transforms
        # 圖片檔案名稱的 list
        self.fnames = []

        for i in range(200):
            self.fnames.append("{:03d}".format(i))

    def __getitem__(self, idx):
        # 利用路徑讀取圖片
        img = Image.open(os.path.join(self.root, self.fnames[idx] + '.png'))
        img = img.filter((ImageFilter.MedianFilter(size=5)))
        # 將輸入的圖片轉換成符合預訓練模型的形式
        img = self.transforms(img)
        # 圖片相對應的 label
        label = self.label[idx]
        return img, label
    
    def __len__(self):
        # 由於已知這次的資料總共有 200 張圖片 所以回傳 200
        return 200

## 載入模型並執行 FGSM 攻擊

In [205]:
class Attacker:
    def __init__(self, img_dir, label, model, label_name):
        self.label_name = label_name
        self.model = model
        self.model.cuda()
        self.model.eval()
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]
        # 把圖片 normalize 到 0~1 之間 mean 0 variance 1
        self.normalize = transforms.Normalize(self.mean, self.std, inplace=False)
        transform = transforms.Compose([                
                        transforms.Resize((224, 224), interpolation=3),
                        transforms.ToTensor(),
                        self.normalize
                    ])
        # 利用 Adverdataset 這個 class 讀取資料
        self.dataset = Adverdataset(img_dir, label, transform)
        
        self.loader = torch.utils.data.DataLoader(
                self.dataset,
                batch_size = 1,
                shuffle = False)

    # FGSM 攻擊
    def fgsm_attack(self, image, epsilon, data_grad):
        # 找出 gradient 的方向
        sign_data_grad = data_grad.sign()
        # 將圖片加上 gradient 方向乘上 epsilon 的 noise
        perturbed_image = image + epsilon * sign_data_grad
        # 將圖片超過 1 或是小於 0 的部分 clip 掉
        # perturbed_image = torch.clamp(perturbed_image, 0, 1)
        return perturbed_image
    
    def attack(self, epsilon):
        # 存下一些成功攻擊後的圖片 以便之後顯示
        adv_examples = []
        wrong, fail, success = 0, 0, 0
        
        times = 0
        for (data, target) in self.loader:
            data, target = data.to(device), target.to(device)
            data_raw = data;
            data.requires_grad = True
            # 將圖片丟入 model 進行測試 得出相對應的 class
            output = self.model(data)
            ####
            ori_output = output
            ####
            init_pred = output.max(1, keepdim=True)[1]
            
            if init_pred.item() != target.item():
                wrong += 1
            else:
                success += 1
        
        print("Success: {}, Wrong: {}, rate: {}".format(success, wrong, success/(success+wrong)))
            # 如果 class 錯誤 就不進行攻擊
            if init_pred.item() != target.item():
                wrong += 1
                adv_ex = data * torch.tensor(self.std, device = device).view(3, 1, 1) + torch.tensor(self.mean, device = device).view(3, 1, 1)
                adv_ex = adv_ex.squeeze().detach().cpu().numpy()
                adv_examples.append( (None, None, None, adv_ex) )
                continue
            
            # 如果 class 正確 就開始計算 gradient 進行 FGSM 攻擊
            loss = F.nll_loss(output, target)
            self.model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            perturbed_data = self.fgsm_attack(data, epsilon, data_grad)
            
            # 再將加入 noise 的圖片丟入 model 進行測試 得出相對應的 class        
            output = self.model(perturbed_data)
            ####
            new_output = output
            ####
            final_pred = output.max(1, keepdim=True)[1]
          
            if final_pred.item() == target.item():
                # 辨識結果還是正確 攻擊失敗
                # Attack again
                output = self.model(perturbed_data)
                loss = F.nll_loss(output, target)
                self.model.zero_grad()
                loss.backward()
                data_grad = data.grad.data
                perturbed_data = self.fgsm_attack(perturbed_data, epsilon, data_grad)
                
                output = self.model(perturbed_data)
                ####
                new_output = output
                ####
                final_pred = output.max(1, keepdim=True)[1]
                
                if final_pred.item() == target.item():
                    fail += 1
                
            else:
                # 辨識結果失敗 攻擊成功
                success += 1
                
            # 將攻擊成功的圖片存入
            adv_ex = perturbed_data * torch.tensor(self.std, device = device).view(3, 1, 1) + torch.tensor(self.mean, device = device).view(3, 1, 1)
            adv_ex = adv_ex.squeeze().detach().cpu().permute(1, 2, 0).numpy() 
            data_raw = data_raw * torch.tensor(self.std, device = device).view(3, 1, 1) + torch.tensor(self.mean, device = device).view(3, 1, 1)
            data_raw = data_raw.squeeze().detach().cpu().numpy()
            adv_examples.append( (init_pred.item(), final_pred.item(), data_raw , adv_ex) )        
            final_acc = (fail / 200 )
            
            
            # Create Q3 Fig
            ori_ex = data * torch.tensor(self.std, device = device).view(3, 1, 1) + torch.tensor(self.mean, device = device).view(3, 1, 1)
            ori_ex = ori_ex.squeeze().detach().cpu().permute(1, 2, 0).numpy() 

            pos_ori = torch.nn.functional.softmax(ori_output).detach().cpu().numpy()[0, :]
            pos_new = torch.nn.functional.softmax(new_output).detach().cpu().numpy()[0, :]
            
            max3_ori = pos_ori.argsort()[-3:][::-1]
            max3_new = pos_new.argsort()[-3:][::-1]
            
            lab3_ori = self.label_name[max3_ori]
            lab3_new = self.label_name[max3_new]

            fig, axs = plt.subplots(2, 2, figsize=(12, 8))
            axs[0][0].imshow(ori_ex)
            axs[0][1].imshow(adv_ex)
            axs[1][0].bar(lab3_ori, pos_ori[max3_ori])
            axs[1][0].set_ylim(0,1)
            axs[1][1].bar(lab3_new, pos_new[max3_new])
            axs[1][1].set_ylim(0,1)

            plt.savefig('../{}.png'.format(times))
            times += 1
            if times == 3:
                break
            # Create Q3 Fig
        print("Epsilon: {}\tTest Accuracy = {} / {} = {}\n".format(epsilon, fail, len(self.loader), final_acc))
    
        return adv_examples, final_acc
        return None, 0

In [206]:
# 載入預訓練模型
model_pool = {
    'vgg16': models.vgg16(pretrained = True),
    'vgg19': models.vgg19(pretrained = True),
    'resnet50': models.resnet50(pretrained = True),
    'resnet101': models.resnet101(pretrained = True),
    'densenet121': models.densenet121(pretrained = True),
    'densenet169': models.densenet169(pretrained = True)
}

## 執行攻擊 並顯示攻擊成功率

In [207]:
if __name__ == '__main__':
    # 讀入圖片相對應的 label
    df = pd.read_csv("./data/labels.csv")
    df = df.loc[:, 'TrueLabel'].to_numpy()
    label_name = pd.read_csv("./data/categories.csv")
    label_name = label_name.loc[:, 'CategoryName'].to_numpy()
    for i in range(len(label_name)):
        label_name[i] = label_name[i].split()[0].split(',')[0]
        
    # new 一個 Attacker class
    model = model_pool['densenet121']
#     attacker = Attacker('./data/images', df, model=model, label_name=label_name)
    attacker = Attacker('../Output/', df, model=model, label_name=label_name)
    # 要嘗試的 epsilon
    epsilons = [0.095]
    accuracies, examples = [], []

    # 進行攻擊 並存起正確率和攻擊成功的圖片
    for eps in epsilons:
        ex, acc = attacker.attack(eps)
        accuracies.append(acc)
        examples.append(ex)

Success: 64, Wrong: 136, rate: 0.32


'ladybug'

## 顯示 FGSM 產生的圖片

In [ ]:
import cv2 

cnt = 0
# plt.figure(figsize=(30, 30))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
#         plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
#         plt.xticks([], [])
#         plt.yticks([], [])
#         if j == 0:
#             plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,orig_img, ex = examples[i][j]
        # plt.title("{} -> {}".format(orig, adv))
#         plt.title("original: {}".format(label_name[orig].split(',')[0]))
#         orig_img = np.transpose(orig_img, (1, 2, 0))
#         plt.imshow(orig_img)
        cnt += 1
#         plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
#         plt.title("adversarial: {}".format(label_name[adv].split(',')[0]))
        
        ex = np.transpose(ex, (1, 2, 0))
                                  
        cv2.imwrite("../Output/{:03d}.png".format(j), ex[:,:, [2,1,0]]*255)
#         plt.imshow(ex)
# plt.tight_layout()
# plt.show()

## Q3

In [17]:
df = pd.read_csv("./data/labels.csv")
label = df.loc[:, 'TrueLabel'].to_numpy()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# 把圖片 normalize 到 0~1 之間 mean 0 variance 1
normalize = transforms.Normalize(mean, std, inplace=False)
transform = transforms.Compose([                
                transforms.Resize((224, 224), interpolation=3),
                transforms.ToTensor(),
                normalize
            ])

dataset = Adverdataset('../Output/', label, transform)
loader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = False)

model = model_pool['densenet121']
model.cuda()
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [19]:
time = 0
for (data, target) in loader:
    data, target = data.to(device), target.to(device)
    data_raw = data;

    output = model(data)
    possibility = torch.nn.functional.softmax(output)
    
    print(target)
    print(possibility[0][304])
    
    break

tensor([305], device='cuda:0')
tensor(0.1357, device='cuda:0', grad_fn=<SelectBackward>)


/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys
